In [127]:
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
np.__version__, pd.__version__

('1.26.4', '2.2.3')

In [128]:
import torch
from torch import nn
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [129]:
class LoanDataset(Dataset):
    def __init__(self, current_slice):
        df = pd.read_csv('data/loan_data.csv')

        # Convert non-numeric columns to numeric
        for col in df.select_dtypes(include=['object']).columns:
            df[col] = df[col].astype('category').cat.codes  # Label encode strings

        # Convert to numpy
        xy = df.to_numpy()
        self.x = torch.from_numpy(xy[current_slice, :-1]).float()  # all columns except last
        self.y = torch.from_numpy(xy[current_slice, [-1]]).float()
        self.n_samples = xy.shape[0]
        
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return self.n_samples
    
split = int(0.8*45000)
trainData = LoanDataset(slice(None,split))
testData = LoanDataset(slice(split,None))
pct = .8
trainDataLoader = DataLoader(dataset=trainData, batch_size=45)
testDataLoader = DataLoader(dataset=testData, batch_size=45)
trainData[0], trainData[4], testData[0]

((tensor([2.2000e+01, 0.0000e+00, 4.0000e+00, 7.1948e+04, 0.0000e+00, 3.0000e+00,
          3.5000e+04, 4.0000e+00, 1.6020e+01, 4.9000e-01, 3.0000e+00, 5.6100e+02,
          0.0000e+00]),
  tensor([1.])),
 (tensor([2.4000e+01, 1.0000e+00, 4.0000e+00, 6.6135e+04, 1.0000e+00, 3.0000e+00,
          3.5000e+04, 3.0000e+00, 1.4270e+01, 5.3000e-01, 4.0000e+00, 5.8600e+02,
          0.0000e+00]),
  tensor([1.])),
 (tensor([2.4000e+01, 1.0000e+00, 3.0000e+00, 3.9534e+04, 4.0000e+00, 2.0000e+00,
          3.5140e+03, 3.0000e+00, 1.4190e+01, 9.0000e-02, 3.0000e+00, 6.2600e+02,
          1.0000e+00]),
  tensor([0.])))

In [130]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.stack = nn.Sequential(
            nn.Linear(13, 20),
            nn.Linear(20, 20),
            nn.Linear(20, 1)
        )
    
    def forward(self, x):
        return self.stack(x)
    
def accuracy(targets, predictions):
    correct = torch.eq(targets, predictions).sum().item()
    return correct / len(targets)

model = Model()
lossFun = nn.BCEWithLogitsLoss()
losses = torch.tensor([])
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
epochs = 100
for epoch in range(epochs):
    trainLoss = 0
    for batch, (X, y) in enumerate(trainDataLoader):
        model.train()
        yPred = model(X)
        loss = lossFun(yPred, y)
        trainLoss += loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch % 10 == 0:
            print(f'{epoch=}, {batch=}')
    trainLoss /= len(trainDataLoader)
    
    testLoss = 0
    model.eval()
    with torch.inference_mode():
        for XTest, yTest in testDataLoader:
            testPred = model(XTest)
            testLoss += lossFun(testPred, yTest)
        testLoss /= len(testDataLoader)
    print(f'{epoch=}, {trainLoss=}, {testLoss=}')

epoch=0, batch=0
epoch=0, batch=10
epoch=0, batch=20
epoch=0, batch=30
epoch=0, batch=40
epoch=0, batch=50
epoch=0, batch=60
epoch=0, batch=70
epoch=0, batch=80
epoch=0, batch=90
epoch=0, batch=100
epoch=0, batch=110
epoch=0, batch=120
epoch=0, batch=130
epoch=0, batch=140
epoch=0, batch=150
epoch=0, batch=160
epoch=0, batch=170
epoch=0, batch=180
epoch=0, batch=190
epoch=0, batch=200
epoch=0, batch=210
epoch=0, batch=220
epoch=0, batch=230
epoch=0, batch=240
epoch=0, batch=250
epoch=0, batch=260
epoch=0, batch=270
epoch=0, batch=280
epoch=0, batch=290
epoch=0, batch=300
epoch=0, batch=310
epoch=0, batch=320
epoch=0, batch=330
epoch=0, batch=340
epoch=0, batch=350
epoch=0, batch=360
epoch=0, batch=370
epoch=0, batch=380
epoch=0, batch=390
epoch=0, batch=400
epoch=0, batch=410
epoch=0, batch=420
epoch=0, batch=430
epoch=0, batch=440
epoch=0, batch=450
epoch=0, batch=460
epoch=0, batch=470
epoch=0, batch=480
epoch=0, batch=490
epoch=0, batch=500
epoch=0, batch=510
epoch=0, batch=520
epoc

IndexError: index 36000 is out of bounds for dimension 0 with size 36000